In [47]:
#-*- coding:utf-8 -*-
import h5py    
import numpy as np  
import os
import pandas as pd
import pickle as pkl
import matplotlib as mpl
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import scipy.stats
from sklearn.cross_decomposition import PLSCanonical, PLSRegression, CCA
# import rcca
import sys
from sklearn.preprocessing import OneHotEncoder

In [2]:
f1 = h5py.File('../data/pbmc_multimodal.h5','r+') 

In [3]:
bio_feature=pd.DataFrame(f1['assays']['ADT']['data'][:],columns=f1['assays']['ADT']['features'][:])
bio_feature=bio_feature[f1['assays']['ADT']['variable.features'][:]]

In [4]:
label1=f1['meta.data']['celltype.l1'][:].T
label2=f1['meta.data']['celltype.l2'][:].T
label3=f1['meta.data']['celltype.l3'][:].T

In [44]:
pd.DataFrame(label1)[0].unique().shape

(8,)

In [45]:
pd.DataFrame(label2)[0].unique().shape

(31,)

In [46]:
pd.DataFrame(label3)[0].unique().shape

(58,)

In [6]:
pd.DataFrame(np.concatenate((label1.reshape(-1,1),label2.reshape(-1,1),label3.reshape(-1,1)),1),columns=['celltype.l1','celltype.l2','celltype.l3'])

,celltype.l1,celltype.l2,celltype.l3
0,b'Mono',b'CD14 Mono',b'CD14 Mono'
1,b'CD4 T',b'CD4 TCM',b'CD4 TCM_1'
2,b'CD8 T',b'CD8 Naive',b'CD8 Naive'
3,b'NK',b'NK',b'NK_2'
4,b'CD8 T',b'CD8 Naive',b'CD8 Naive'
...,...,...,...
161759,b'CD8 T',b'CD8 Naive',b'CD8 Naive'
161760,b'Mono',b'CD14 Mono',b'CD14 Mono'
161761,b'B',b'B intermediate',b'B intermediate kappa'
161762,b'Mono',b'CD16 Mono',b'CD16 Mono'


In [7]:
enc = OneHotEncoder(handle_unknown='ignore')

In [8]:
enc.fit(label1.reshape(-1,1))

OneHotEncoder(handle_unknown='ignore')

In [9]:
type_label1=enc.categories_

In [10]:
label1_one_hot=enc.transform(label1.reshape(-1,1)).toarray()

In [11]:
enc.fit(label2.reshape(-1,1))
type_label2=enc.categories_
label2_one_hot=enc.transform(label2.reshape(-1,1)).toarray()

In [12]:
enc.fit(label3.reshape(-1,1))
type_label3=enc.categories_
label3_one_hot=enc.transform(label3.reshape(-1,1)).toarray()

In [13]:
type_label=np.concatenate((type_label1,type_label2,type_label3),1).reshape(-1)

In [14]:
type_label

array([b'B', b'CD4 T', b'CD8 T', b'DC', b'Mono', b'NK', b'other',
       b'other T', b'ASDC', b'B intermediate', b'B memory', b'B naive',
       b'CD14 Mono', b'CD16 Mono', b'CD4 CTL', b'CD4 Naive',
       b'CD4 Proliferating', b'CD4 TCM', b'CD4 TEM', b'CD8 Naive',
       b'CD8 Proliferating', b'CD8 TCM', b'CD8 TEM', b'Doublet', b'Eryth',
       b'HSPC', b'ILC', b'MAIT', b'NK', b'NK Proliferating',
       b'NK_CD56bright', b'Plasmablast', b'Platelet', b'Treg', b'cDC1',
       b'cDC2', b'dnT', b'gdT', b'pDC', b'ASDC_mDC', b'ASDC_pDC',
       b'B intermediate kappa', b'B intermediate lambda',
       b'B memory kappa', b'B memory lambda', b'B naive kappa',
       b'B naive lambda', b'CD14 Mono', b'CD16 Mono', b'CD4 CTL',
       b'CD4 Naive', b'CD4 Proliferating', b'CD4 TCM_1', b'CD4 TCM_2',
       b'CD4 TCM_3', b'CD4 TEM_1', b'CD4 TEM_2', b'CD4 TEM_3',
       b'CD4 TEM_4', b'CD8 Naive', b'CD8 Naive_2', b'CD8 Proliferating',
       b'CD8 TCM_1', b'CD8 TCM_2', b'CD8 TCM_3', b'CD8 TEM_1',
  

In [15]:
label_one_hot=np.concatenate((label1_one_hot,label2_one_hot,label3_one_hot),1)

In [16]:
label_one_hot_df=pd.DataFrame(label_one_hot,columns=type_label)

In [17]:
label_one_hot_df

,b'B',b'CD4 T',b'CD8 T',b'DC',b'Mono',b'NK',b'other',b'other T',b'ASDC',b'B intermediate',...,b'cDC1',b'cDC2_1',b'cDC2_2',b'dnT_1',b'dnT_2',b'gdT_1',b'gdT_2',b'gdT_3',b'gdT_4',b'pDC'
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161759,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161760,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161761,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161762,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
label_one_hot_df.std()

b'B'        0.279343
b'CD4 T'    0.434994
b'CD8 T'    0.364221
b'DC'       0.147291
b'Mono'     0.459545
              ...   
b'gdT_1'    0.099966
b'gdT_2'    0.068784
b'gdT_3'    0.066751
b'gdT_4'    0.056769
b'pDC'      0.072762
Length: 97, dtype: float64

In [ ]:
label_one_hot_df['B']=label_one_hot_df['B']-label_one_hot_df['B'].mean()
label_one_hot_df['B']=label_one_hot_df['B']/label_one_hot_df['B'].std()

In [ ]:
label_one_hot_df['B'].std()

In [ ]:
label_one_hot_df['B'].mean()

In [ ]:
bio_feature

In [19]:
for label in label_one_hot_df.columns:
    label_one_hot_df[label]=label_one_hot_df[label]-label_one_hot_df[label].mean()
    label_one_hot_df[label]=label_one_hot_df[label]/label_one_hot_df[label].std()

In [20]:
for label in bio_feature.columns:
    bio_feature[label]=bio_feature[label]-bio_feature[label].mean()
    bio_feature[label]=bio_feature[label]/bio_feature[label].std()

In [21]:
label_one_hot_df

,b'B',b'CD4 T',b'CD8 T',b'DC',b'Mono',b'NK',b'other',b'other T',b'ASDC',b'B intermediate',...,b'cDC1',b'cDC2_1',b'cDC2_2',b'dnT_1',b'dnT_2',b'gdT_1',b'gdT_2',b'gdT_3',b'gdT_4',b'pDC'
0,-0.305394,-0.582678,-0.432280,-0.150632,1.516779,-0.361145,-0.147446,-0.209301,-0.02168,-0.123520,...,-0.030567,-0.069248,-0.103941,-0.034201,-0.032147,-0.100984,-0.069112,-0.06705,-0.056952,-0.073151
1,-0.305394,1.716202,-0.432280,-0.150632,-0.659288,-0.361145,-0.147446,-0.209301,-0.02168,-0.123520,...,-0.030567,-0.069248,-0.103941,-0.034201,-0.032147,-0.100984,-0.069112,-0.06705,-0.056952,-0.073151
2,-0.305394,-0.582678,2.313304,-0.150632,-0.659288,-0.361145,-0.147446,-0.209301,-0.02168,-0.123520,...,-0.030567,-0.069248,-0.103941,-0.034201,-0.032147,-0.100984,-0.069112,-0.06705,-0.056952,-0.073151
3,-0.305394,-0.582678,-0.432280,-0.150632,-0.659288,2.768956,-0.147446,-0.209301,-0.02168,-0.123520,...,-0.030567,-0.069248,-0.103941,-0.034201,-0.032147,-0.100984,-0.069112,-0.06705,-0.056952,-0.073151
4,-0.305394,-0.582678,2.313304,-0.150632,-0.659288,-0.361145,-0.147446,-0.209301,-0.02168,-0.123520,...,-0.030567,-0.069248,-0.103941,-0.034201,-0.032147,-0.100984,-0.069112,-0.06705,-0.056952,-0.073151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161759,-0.305394,-0.582678,2.313304,-0.150632,-0.659288,-0.361145,-0.147446,-0.209301,-0.02168,-0.123520,...,-0.030567,-0.069248,-0.103941,-0.034201,-0.032147,-0.100984,-0.069112,-0.06705,-0.056952,-0.073151
161760,-0.305394,-0.582678,-0.432280,-0.150632,1.516779,-0.361145,-0.147446,-0.209301,-0.02168,-0.123520,...,-0.030567,-0.069248,-0.103941,-0.034201,-0.032147,-0.100984,-0.069112,-0.06705,-0.056952,-0.073151
161761,3.274441,-0.582678,-0.432280,-0.150632,-0.659288,-0.361145,-0.147446,-0.209301,-0.02168,8.095786,...,-0.030567,-0.069248,-0.103941,-0.034201,-0.032147,-0.100984,-0.069112,-0.06705,-0.056952,-0.073151
161762,-0.305394,-0.582678,-0.432280,-0.150632,1.516779,-0.361145,-0.147446,-0.209301,-0.02168,-0.123520,...,-0.030567,-0.069248,-0.103941,-0.034201,-0.032147,-0.100984,-0.069112,-0.06705,-0.056952,-0.073151


In [22]:
label_one_hot_df.astype('float32').to_csv('label_one_hot_df.csv',index=None)

In [23]:
bio_feature.astype('float32').to_csv('bio_feature',index=None)

In [ ]:
label_one_hot_df[:10000].astype('float32').to_csv('label_one_hot_toy.csv',index=None)
bio_feature[:10000].astype('float32').to_csv('bio_feature_toy.csv',index=None)

In [26]:
kf = KFold(n_splits=10,shuffle=True)

In [27]:
for train_index, test_index in kf.split(label_one_hot_df):
    print(train_index,test_index)

[     0      1      2 ... 161761 161762 161763] [     6     11     16 ... 161720 161726 161739]
[     0      1      2 ... 161761 161762 161763] [     5     15     33 ... 161734 161748 161752]
[     0      1      2 ... 161760 161761 161763] [     4     17     39 ... 161745 161753 161762]
[     0      1      2 ... 161761 161762 161763] [     8     21     27 ... 161729 161732 161755]
[     0      1      2 ... 161761 161762 161763] [    28     35     41 ... 161744 161754 161758]
[     0      1      3 ... 161758 161760 161762] [     2     20     24 ... 161759 161761 161763]
[     0      1      2 ... 161761 161762 161763] [    34     51     54 ... 161674 161743 161751]
[     0      1      2 ... 161761 161762 161763] [    12     23     36 ... 161679 161681 161719]
[     0      1      2 ... 161761 161762 161763] [     3      9     10 ... 161740 161742 161760]
[     2      3      4 ... 161761 161762 161763] [     0      1      7 ... 161749 161750 161757]


In [28]:
for train_index, test_index in kf.split(bio_feature):
    print(train_index,test_index)

[     0      1      3 ... 161761 161762 161763] [     2      8     21 ... 161745 161752 161755]
[     0      2      3 ... 161761 161762 161763] [     1     28     41 ... 161725 161734 161754]
[     0      1      2 ... 161761 161762 161763] [     7     12     17 ... 161731 161742 161743]
[     0      1      2 ... 161761 161762 161763] [     5     18     32 ... 161751 161759 161760]
[     0      1      2 ... 161760 161761 161763] [    10     13     16 ... 161744 161748 161762]
[     0      1      2 ... 161760 161761 161762] [     6      9     24 ... 161722 161736 161763]
[     1      2      3 ... 161761 161762 161763] [     0     19     45 ... 161749 161756 161757]
[     0      1      2 ... 161761 161762 161763] [    11     14     22 ... 161733 161746 161753]
[     0      1      2 ... 161761 161762 161763] [     3     15     33 ... 161698 161707 161708]
[     0      1      2 ... 161760 161762 161763] [     4     25     36 ... 161741 161758 161761]


In [33]:
label_one_hot_df.loc[train_index.tolist()].astype('float32').to_csv('label_one_hot_train.csv',index=None)

In [34]:
bio_feature.loc[train_index.tolist()].astype('float32').to_csv('bio_feature_train.csv',index=None)

In [35]:
label_one_hot_df.loc[test_index.tolist()].astype('float32').to_csv('label_one_hot_test.csv',index=None)

In [36]:
bio_feature.loc[test_index.tolist()].astype('float32').to_csv('bio_feature_test.csv',index=None)

In [38]:
pd.DataFrame(label1).loc[test_index.tolist()].to_csv('label1_test.csv',index=None)

In [39]:
pd.DataFrame(label2).loc[test_index.tolist()].to_csv('label2_test.csv',index=None)

In [40]:
pd.DataFrame(label3).loc[test_index.tolist()].to_csv('label3_test.csv',index=None)

In [49]:
label_one_hot_test=pd.read_csv('label_one_hot_test.csv')

In [52]:
bio_feature_test=pd.read_csv('bio_feature_test.csv')

In [53]:
bio_feature_test

,b'CD80',b'CD86',b'CD274',b'CD273',b'CD275-1',b'CD11b-1',b'Galectin-9',b'CD270',b'CD252',b'CD155',...,b'CD161',b'CCR10',b'CD271',b'GP130',b'CD199',b'CD45RB',b'CD46',b'VEGFR-3',b'CLEC2',b'CD26-2'
0,0.285630,-0.703389,1.837432,1.405582,0.335990,-0.839572,0.733351,0.889074,0.742269,0.043585,...,-0.883260,0.844059,-0.690680,1.075548,-1.239343,1.745541,2.502320,-0.883154,-0.162051,0.069817
1,-1.241127,-0.676597,-1.573479,0.328211,-0.080189,-0.736324,2.282444,1.011919,-0.489794,0.087412,...,0.062112,0.186817,-0.893517,-0.115105,0.066639,1.058168,1.427678,-0.852336,-0.792224,0.295600
2,-0.706665,2.052312,-0.182726,-0.198669,0.114327,1.402590,-0.818671,0.204490,-0.780894,2.362728,...,-0.883260,0.836663,-0.717080,-0.135968,-0.569780,-1.384825,0.676840,0.566667,0.072012,-0.605086
3,0.502179,-0.623043,-0.899412,1.130993,0.786396,-0.536108,1.100265,1.253220,0.341666,-0.066114,...,-0.496579,1.837569,-1.830130,1.880790,-0.615603,1.685815,2.583369,1.341764,-0.895068,1.252755
4,-0.295936,-0.431617,0.243045,-0.275788,-0.631440,-0.127548,1.146044,0.616428,-0.399482,-0.977134,...,1.671190,1.226866,1.153487,-1.347116,-2.246066,0.247598,-0.313463,1.381202,-0.373281,-0.034047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16171,1.032937,-0.676100,1.298115,-0.007828,-0.808322,-0.841402,0.609796,0.369638,-1.277208,-0.521076,...,-0.369392,-0.528452,0.592914,-1.204046,-0.348223,1.735887,-0.083116,1.531193,-0.468647,1.349969
16172,-0.122626,-0.502170,-1.573479,-0.115604,-0.531594,0.158917,-0.915498,-1.614799,-0.238542,-0.211414,...,-0.020476,-0.340004,1.010097,-0.378809,0.823893,-0.109162,0.162012,-0.645070,0.036330,-0.365812
16173,-1.241127,-0.648932,-0.635537,0.045903,-0.789140,-0.340506,-0.147362,0.476085,0.926403,-0.977134,...,0.489640,-1.955705,-0.469952,0.893117,-0.236106,-1.163846,-0.588903,0.315932,2.264911,-0.583276
16174,2.543858,-0.470278,-0.725209,-0.068374,0.462054,-0.866204,-0.841436,-0.885993,-0.191088,-0.545265,...,-0.396646,-0.621811,-0.939027,1.468695,-0.071479,1.072803,-1.195820,-0.605924,-0.487349,0.733342


In [55]:
kf = KFold(n_splits=5,shuffle=True)
for train_index, test_index in kf.split(bio_feature_test):
    print(train_index,test_index)

[    0     2     3 ... 16172 16173 16174] [    1    13    28 ... 16166 16167 16175]
[    1     2     5 ... 16173 16174 16175] [    0     3     4 ... 16168 16169 16170]
[    0     1     2 ... 16172 16174 16175] [   16    17    19 ... 16162 16164 16173]
[    0     1     2 ... 16171 16173 16175] [    6     8    23 ... 16157 16172 16174]
[    0     1     3 ... 16173 16174 16175] [    2     5     7 ... 16163 16165 16171]


In [56]:
label_one_hot_test.loc[test_index.tolist()].astype('float32').to_csv('label_one_hot_classification_test.csv',index=None)
bio_feature_test.loc[test_index.tolist()].astype('float32').to_csv('bio_feature_classification_test.csv',index=None)

In [57]:
label_one_hot_test.loc[train_index.tolist()].astype('float32').to_csv('label_one_hot_classification_train.csv',index=None)
bio_feature_test.loc[train_index.tolist()].astype('float32').to_csv('bio_feature_classification_train.csv',index=None)